In [ ]:
# Import libraries
import os
import numpy as np 
import pandas as pd 
import tensorflow
import random
import cv2
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
import warnings
from keras import backend as K
from sklearn.model_selection import train_test_split
from PIL import Image
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from sklearn.metrics import confusion_matrix
from keras.models import Model
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.models import load_model

In [ ]:
def load_images_from_folder(folder):
	images = []
	dirs = os.listdir(folder)

	for filename in dirs:
		if os.path.isfile(folder+filename):
			im = Image.open(folder+filename)
			imResize = im.resize((224,224), Image.ANTIALIAS)
			imResize = np.array(imResize)
			if imResize is not None:
				images.append(imResize)
	return images


def prepareData(parentPath):
	
	# Make a list of all the 0 label and 1 label images for train, val, and test sets

	all0_path = list()
	

	all1_path = list()
	

	all0_path.append(parentPath+'/0/')
	all1_path.append(parentPath+'/1/')


	# Read images into respective lists
	

	allX = list()
	allY = list()
	xTrain = list()
	yTrain = list()
	xVal = list()
	yVal = list()
	xTest = list()
	yTest = list()

	# Prepare all data for 0 class
	print('\n\n Class 0, reading started..\n\n')
	

	tempImgs = list()
	tempImgs = load_images_from_folder(all0_path[0])
	for i in range(len(tempImgs)):
		allX.append(tempImgs[i])
		allY.append(0)

	# Prepare all data for 1 class
	print('\n\n Class 1, reading started..\n\n')
	tempImgs = list()
	tempImgs = load_images_from_folder(all1_path[0])
	for i in range(len(tempImgs)):
		allX.append(tempImgs[i])
		allY.append(1)

	xTrain,xTest,yTrain,yTest = train_test_split(allX,allY, test_size=0.20, random_state=23, shuffle = True)

	xpTrain,xVal,ypTrain,yVal = train_test_split(xTrain,yTrain, test_size=0.10, random_state=23, shuffle = True)

	return xpTrain, ypTrain, xVal, yVal, xTest, yTest

In [ ]:
from keras.applications.densenet import DenseNet121
def DenseNet121_transfer_actual(input_shape):
  res = DenseNet121(weights=None, include_top=False, input_shape=input_shape)
  for layers in res.layers:
    layers.trainable = True

  model = Sequential()
  model.add(res)
  
  model.add(Flatten())

  ## The following is the vanilla VGG16 architecture's FC layers
  model.add(Dense(units=4096,activation="relu"))
  model.add(Dense(units=4096,activation="relu"))
  model.add(Dense(units=1000,activation="relu"))
  model.add(Dense(units=1, activation="sigmoid"))

  opt = SGD(lr=0.001, momentum=0.9)
  model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
  model.summary()
  return model

In [ ]:
from keras.applications.resnet import ResNet50
def ResNet50_transfer_actual(input_shape):
  res = ResNet50(weights=None, include_top=False, input_shape=input_shape)
  for layers in res.layers:
    layers.trainable = True

  model = Sequential()
  model.add(res)
  
  model.add(Flatten())

  ## The following is the vanilla VGG16 architecture's FC layers
  model.add(Dense(units=4096,activation="relu"))
  model.add(Dense(units=4096,activation="relu"))
  model.add(Dense(units=1000,activation="relu"))
  model.add(Dense(units=1, activation="sigmoid"))

  opt = SGD(lr=0.001, momentum=0.9)
  model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
  model.summary()
  return model

In [ ]:
from keras.applications.inception_v3 import InceptionV3
def InceptionV3_transfer_actual(input_shape):
  res = InceptionV3(weights=None, include_top=False, input_shape=input_shape)
  for layers in res.layers:
    layers.trainable = True

  model = Sequential()
  model.add(res)
  
  model.add(Flatten())

  ## The following is the vanilla VGG16 architecture's FC layers
  model.add(Dense(units=4096,activation="relu"))
  model.add(Dense(units=4096,activation="relu"))
  model.add(Dense(units=1000,activation="relu"))
  model.add(Dense(units=1, activation="sigmoid"))

  opt = SGD(lr=0.001, momentum=0.9)
  model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
  model.summary()
  return model

In [ ]:
import tensorflow as tf
from sklearn.metrics import roc_auc_score

def auroc(y_true, y_pred):
  return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

In [ ]:
parentPath = '/content/drive/My Drive/Colab Notebooks/DFU-Dataset2'
xTrain, yTrain, xVal, yVal, xTest, yTest = prepareData(parentPath)
xTrain = np.array(xTrain)
yTrain = np.array(yTrain)
xVal = np.array(xVal)
yVal = np.array(yVal)
xTest = np.array(xTest)
yTest = np.array(yTest)

# Prepare the data
img_width, img_height = 224, 224

if K.image_data_format() == 'channels_first': 
  input_shape = (3, img_width, img_height) 
else:
  input_shape = (img_width, img_height, 3) 

# normalize inputs from 0-255 to 0-1
xTrain = xTrain / 255
xVal = xVal / 255
xTest = xTest / 255


In [ ]:
model1 = InceptionV3_transfer_actual(input_shape)
#model2 = ResNet50_transfer_actual(input_shape)
#model3 = DenseNet121_transfer_actual(input_shape)

## Train Model
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=200, restore_best_weights=True)
model1.fit(xTrain, yTrain, epochs=1000, batch_size=50, validation_data=(xVal, yVal), verbose=1, callbacks = [es])
#model2.fit(xTrain, yTrain, epochs=1000, batch_size=50, validation_data=(xVal, yVal), verbose=1, callbacks = [es])
#model3.fit(xTrain, yTrain, epochs=1000, batch_size=50, validation_data=(xVal, yVal), verbose=1, callbacks = [es])
  
model1.save('inception_v3.h5')
#model2.save('resnet_50.h5')
#model3.save('densenet121.h5')

In [ ]:
model1 = load_model('inception_v3.h5')
testPreds1 = model1.predict_classes(xTest)
np.save('testPreds1',testPreds1)
print(testPreds1)
print("done\n")

In [ ]:
model1 = load_model('resnet_50.h5')
testPreds2 = model1.predict_classes(xTest)
np.save('testPreds2',testPreds2)
print(testPreds2)
print("done\n")

In [ ]:
model1 = load_model('densenet121.h5')
testPreds3 = model1.predict_classes(xTest)
np.save('testPreds3',testPreds3)
print(testPreds3)
print("done\n")

In [ ]:
'''
model1 = load_model('inception_v3.h5')
testPreds1 = model1.predict_classes(xTest)
print("done\n")
model1 = load_model('resnet_50.h5')
testPreds2 = model1.predict_classes(xTest)
print("done\n")
model1 = load_model('densenet.h5')
testPreds3 = model1.predict_classes(xTest)
print("done\n")
'''
testPreds1 = np.load('testPreds1.npy')
testPreds2 = np.load('testPreds2.npy')
testPreds3 = np.load('testPreds3.npy')

testPreds = []

length = len(testPreds1)
print(length)

for i in range(length):
  total = testPreds1[i] + testPreds2[i] + testPreds3[i]
  if(total>1):
      testPreds.append(1)
  else:
      testPreds.append(0)

cm = confusion_matrix(yTest, testPreds)
cm1 = confusion_matrix(yTest, testPreds1)
cm2 = confusion_matrix(yTest, testPreds2)
cm3 = confusion_matrix(yTest, testPreds3)

sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
precision = cm[0, 0] / (cm[0, 0] + cm[1, 0])
recall = cm[1, 1] / (cm[0, 1] + cm[1, 1])
accuracy = (cm[0,0] + cm[1,1]) / (cm[0,0] + cm[1,1] + cm[0,1] + cm[1,0])

sensitivity1 = cm1[0, 0] / (cm1[0, 0] + cm1[0, 1])
specificity1 = cm1[1, 1] / (cm1[1, 0] + cm1[1, 1])
precision1 = cm1[0, 0] / (cm1[0, 0] + cm1[1, 0])
recall1 = cm1[1, 1] / (cm1[0, 1] + cm1[1, 1])
accuracy1 = (cm1[0,0] + cm1[1,1]) / (cm1[0,0] + cm1[1,1] + cm1[0,1] + cm1[1,0])

sensitivity2 = cm2[0, 0] / (cm2[0, 0] + cm2[0, 1])
specificity2 = cm2[1, 1] / (cm2[1, 0] + cm2[1, 1])
precision2 = cm2[0, 0] / (cm2[0, 0] + cm2[1, 0])
recall2 = cm2[1, 1] / (cm2[0, 1] + cm2[1, 1])
accuracy2 = (cm2[0,0] + cm2[1,1]) / (cm2[0,0] + cm2[1,1] + cm2[0,1] + cm2[1,0])

sensitivity3 = cm3[0, 0] / (cm3[0, 0] + cm3[0, 1])
specificity3 = cm3[1, 1] / (cm3[1, 0] + cm3[1, 1])
precision3 = cm3[0, 0] / (cm3[0, 0] + cm3[1, 0])
recall3 = cm3[1, 1] / (cm3[0, 1] + cm3[1, 1])
accuracy3 = (cm3[0,0] + cm3[1,1]) / (cm3[0,0] + cm3[1,1] + cm3[0,1] + cm3[1,0])

print("Seed = 5")
print()
print("For ensemble : ")
print()
print("\n Accuracy: ",accuracy)
print()
print("\n Sensitivity: ",sensitivity)
print()
print("\n Specificity: ",specificity)
print()
print("\n Precision: ",precision)
print()
print("\n Recall: ",recall)
print()
print("AUC Value: ", roc_auc_score(yTest, testPreds))

print("For Inception : ")
print()
print("\n Accuracy: ",accuracy1)
print()
print("\n Sensitivity: ",sensitivity1)
print()
print("\n Specificity: ",specificity1)
print()
print("\n Precision: ",precision1)
print()
print("\n Recall: ",recall1)
print()
print("AUC Value: ", roc_auc_score(yTest, testPreds1))

print("For ResNet : ")
print()
print("\n Accuracy: ",accuracy2)
print()
print("\n Sensitivity: ",sensitivity2)
print()
print("\n Specificity: ",specificity2)
print()
print("\n Precision: ",precision2)
print()
print("\n Recall: ",recall2)
print()
print("AUC Value: ", roc_auc_score(yTest, testPreds2))

print("For DenseNet : ")
print()
print("\n Accuracy: ",accuracy3)
print()
print("\n Sensitivity: ",sensitivity3)
print()
print("\n Specificity: ",specificity3)
print()
print("\n Precision: ",precision3)
print()
print("\n Recall: ",recall3)
print()
print("AUC Value: ", roc_auc_score(yTest, testPreds3))


In [ ]:

def main():

  ## First read and organize the data into train-dev-test sets.

  parentPath = '/content/drive/My Drive/Colab Notebooks/DFU-Dataset2'

  xTrain, yTrain, xVal, yVal, xTest, yTest = prepareData(parentPath)

  xTrain = np.array(xTrain)
  yTrain = np.array(yTrain)
  xVal = np.array(xVal)
  yVal = np.array(yVal)
  xTest = np.array(xTest)
  yTest = np.array(yTest)

  # Prepare the data
  img_width, img_height = 224, 224
  
  if K.image_data_format() == 'channels_first': 
    input_shape = (3, img_width, img_height) 
  else:
    input_shape = (img_width, img_height, 3) 
  
  # normalize inputs from 0-255 to 0-1
  xTrain = xTrain / 255
  xVal = xVal / 255
  xTest = xTest / 255

  ## Define model

  model1 = InceptionV3_transfer_actual(input_shape)
  model2 = ResNet50_transfer_actual(input_shape)
  model3 = DenseNet121_transfer_actual(input_shape)

  ## Train Model
  es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=200, restore_best_weights=True)
  #model1.fit(xTrain, yTrain, epochs=10, batch_size=50, validation_data=(xVal, yVal), verbose=1, callbacks = [es])
  #model2.fit(xTrain, yTrain, epochs=10, batch_size=50, validation_data=(xVal, yVal), verbose=1, callbacks = [es])
  #model3.fit(xTrain, yTrain, epochs=10, batch_size=50, validation_data=(xVal, yVal), verbose=1, callbacks = [es])
  
  #model1.save('inception_v3.h5')
  #model2.save('resnet_50.h5')
  #model3.save('densenet121.h5')
  # to load the saved model: model = load_model('my_model.h5') 
  ## evaluate model on test data

  
  model1 = load_model('inception_v3.h5')
  testPreds1 = model1.predict_classes(xTest)
  print("done\n")
  model1 = load_model('resnet_50.h5')
  testPreds2 = model1.predict_classes(xTest)
  print("done\n")
  model1 = load_model('densenet121.h5')
  testPreds3 = model1.predict_classes(xTest)
  print("done\n")
  
  testPreds1 = model1.predict_classes(xTest)
  print("done\n")
  testPreds2 = model2.predict_classes(xTest)
  print("done\n")
  testPreds3 = model3.predict_classes(xTest)
  print("done\n")
  

  testPreds = []

  length = len(testPreds1)
  print(length)

  for i in range(length):
    total = testPreds1[i] + testPreds2[i] + testPreds3[i]
    if(total>1):
       testPreds.append(1)
    else:
       testPreds.append(0)

  
  _, acc = model.evaluate(xTrain, yTrain, verbose=0)
  print('Training accuracy> %.3f' % (acc * 100.0))
  _, acc = model.evaluate(xVal, yVal, verbose=0)
  print('Validation accuracy> %.3f' % (acc * 100.0))
  _, acc = model.evaluate(xTest, yTest, verbose=0)
  print('Testing accuracy> %.3f' % (acc * 100.0))
  testPreds = model.predict_classes(xTest)
  
  
  #print(testPreds);
  cm = confusion_matrix(yTest, testPreds)
  sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
  specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
  precision = cm[0, 0] / (cm[0, 0] + cm[1, 0])
  recall = cm[1, 1] / (cm[0, 1] + cm[1, 1])
  accuracy = (cm[0,0] + cm[1,1]) / (cm[0,0] + cm[1,1] + cm[0,1] + cm[1,0])

  print("\n Accuracy: ",accuracy)
  print()
  print("\n Sensitivity: ",sensitivity)
  print()
  print("\n Specificity: ",specificity)
  print()
  print("\n Precision: ",precision)
  print()
  print("\n Recall: ",recall)
  print()
  print("AUC Value: ", roc_auc_score(yTest, testPreds))

  ## Visualize Results

main()


In [ ]:
import statistics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt

pred = testPreds
label = yTest[:]
fpr, tpr, thresh = metrics.roc_curve(label, pred)
auc = metrics.roc_auc_score(label, pred)
plt.plot(fpr,tpr,label="Ensemble, auc="+str(auc))

pred = testPreds1
label = yTest[:]
fpr, tpr, thresh = metrics.roc_curve(label, pred)
auc = metrics.roc_auc_score(label, pred)
plt.plot(fpr,tpr,label="InceptionV3, auc="+str(auc))

pred = testPreds2
label = yTest[:]
fpr, tpr, thresh = metrics.roc_curve(label, pred)
auc = metrics.roc_auc_score(label, pred)
plt.plot(fpr,tpr,label="ResNet50, auc="+str(auc))

pred = testPreds3
label = yTest[:]
fpr, tpr, thresh = metrics.roc_curve(label, pred)
auc = metrics.roc_auc_score(label, pred)
plt.plot(fpr,tpr,label="DenseNet121, auc="+str(auc))

plt.title('ROC Curve: DFU v/s No DFU', fontsize = 12)
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.legend(loc=0)
#plt.show()
plt.savefig('ROC_DFU.eps', format = 'eps')